<a href="https://colab.research.google.com/github/eman-kom/tensorflow-fundamentals/blob/main/08_NLP_introduction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-a75bec62-f416-2dd9-5323-99f6bf231ce7)


In [2]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

--2022-07-07 06:35:03--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2022-07-07 06:35:03 (79.5 MB/s) - ‘helper_functions.py’ saved [10246/10246]



In [3]:
from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys

In [4]:
!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip

unzip_data("nlp_getting_started.zip")

--2022-07-07 06:35:06--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.251.111.128, 142.251.16.128, 142.250.81.208, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.251.111.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.004s  

2022-07-07 06:35:06 (154 MB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



In [5]:
import pandas as pd

train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [6]:
train_df_shuffled = train_df.sample(frac=1, random_state=42)
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [7]:
train_df.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [8]:
len(train_df), len(test_df)

(7613, 3263)

In [9]:
import random
random_index = random.randint(0, len(train_df)-5)

for row in train_df_shuffled[["text", "target"]][random_index:random_index+5].itertuples():
  _, text, target = row
  print(f"Target: {target}", "<real disaster>" if target > 0 else "<not a real disaster>")
  print(f"Text\n{text}\n")
  print("---\n")

Target: 0 <not a real disaster>
Text
CAMILA'S DOING A FOLLOW SPREE TONIGHT IM SCREAMING OF HAPPINESS

---

Target: 0 <not a real disaster>
Text
Pandemonium In Aba As Woman Delivers Baby Without Face (Photos) - http://t.co/Ykdsp0nRDQ

---

Target: 1 <real disaster>
Text
@rvfriedmann Hell is just a fraction of his belief of total annihilation destruction of USA @LodiSilverado @ritzy_jewels

---

Target: 1 <real disaster>
Text
Islamic State group in Egypt threatens to kill Croat hostage http://t.co/VdgfXYX3bw

---

Target: 0 <not a real disaster>
Text
@HomeworldGym @thisisperidot D: What? That's a tragedy. You have a wonderful nose

---



In [10]:
from sklearn.model_selection import train_test_split

train_sentences, val_sentences, train_labels, val_labels = train_test_split(
    train_df_shuffled["text"].to_numpy(),
    train_df_shuffled["target"].to_numpy(),
    test_size=0.1,
    random_state=42
)

In [11]:
train_sentences[:10], train_labels[:10]

(array(['@mogacola @zamtriossu i screamed after hitting tweet',
        'Imagine getting flattened by Kurt Zouma',
        '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
        "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
        'Somehow find you and I collide http://t.co/Ee8RpOahPk',
        '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
        'destroy the free fandom honestly',
        'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
        '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
        'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
       dtype=object), array([0, 

In [12]:
round(sum([len(i.split()) for i in train_sentences])/len(train_sentences))

15

In [13]:
import tensorflow as tf
from tensorflow.keras.layers import TextVectorization

text_vectorizer = TextVectorization(
    max_tokens=10000,
    output_mode="int",
    output_sequence_length=15,
)

In [14]:
text_vectorizer.adapt(train_sentences)

In [15]:
text_vectorizer(["There's a flood in my street!"])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[264,   3, 232,   4,  13, 698,   0,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [16]:
random_sent = random.choice(train_sentences)
print(f"Before: {random_sent}")
text_vectorizer([random_sent])

Before: Hellfire! We donÛªt even want to think about it or mention it so letÛªs not do anything that leads to it!


<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[ 546,   46, 1029,  151,  138,    5,  125,   54,   15,   53, 1636,
          15,   28, 2405,   34]])>

In [17]:
words_in_vocab = text_vectorizer.get_vocabulary()
top_5_words = words_in_vocab[:5]
bottom_5_words = words_in_vocab[-5:]
len(words_in_vocab), top_5_words, bottom_5_words

(10000,
 ['', '[UNK]', 'the', 'a', 'in'],
 ['pages', 'paeds', 'pads', 'padres', 'paddytomlinson1'])

In [18]:
from tensorflow.keras import layers

embedding = layers.Embedding(
    input_dim=10000,
    output_dim=128,
    input_length=15
)

embedding

In [19]:
sample_embed = embedding(text_vectorizer([random.choice(train_sentences)]))
sample_embed

<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[-0.04727489,  0.00413039,  0.02918592, ...,  0.02587075,
         -0.0399493 ,  0.00582303],
        [ 0.00371378,  0.00729749,  0.02096326, ..., -0.00671252,
         -0.01839857, -0.04495699],
        [-0.01662181, -0.04094366,  0.02272885, ..., -0.01341771,
          0.04950306,  0.04909046],
        ...,
        [-0.02197913,  0.03348363,  0.04861161, ..., -0.00827495,
          0.01822697, -0.04882536],
        [-0.02197913,  0.03348363,  0.04861161, ..., -0.00827495,
          0.01822697, -0.04882536],
        [-0.02197913,  0.03348363,  0.04861161, ..., -0.00827495,
          0.01822697, -0.04882536]]], dtype=float32)>

# Model 0: Baseline Model -> Sklearn Multinomial Naive Bayes using TF-IDF

In [20]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

model_0 = Pipeline([
    ("tfidf", TfidfVectorizer()),
    ("clf", MultinomialNB())
])

model_0.fit(train_sentences, train_labels)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [21]:
baseline_score = model_0.score(val_sentences, val_labels)
print(f"Baseline Score: {baseline_score*100:.2f}%")

Baseline Score: 79.27%


In [22]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def calculate_results(y_true, y_pred):
  model_accuracy = accuracy_score(y_true, y_pred) * 100
  model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred, average="weighted")
  model_results = {
      "accuracy": model_accuracy,
      "precision": model_precision,
      "recall": model_recall,
      "f1": model_f1
  }

  return model_results

In [23]:
baseline_preds = model_0.predict(val_sentences)
baseline_preds[:5]

array([1, 1, 1, 0, 0])

In [24]:
baseline_results = calculate_results(val_labels, baseline_preds)
baseline_results

{'accuracy': 79.26509186351706,
 'f1': 0.7862189758049549,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706}

In [25]:
from helper_functions import create_tensorboard_callback

SAVE_DIR = "model_logs"

# Model 1: Simple Dense Model

In [26]:
from tensorflow.keras import layers

inputs = layers.Input(shape=(1,), dtype=tf.string)
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.GlobalAveragePooling1D()(x)
outputs = layers.Dense(1, activation="sigmoid")(x)

model_1 = tf.keras.Model(inputs, outputs, name="model_1_dense")
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization (TextVec  (None, 15)               0         
 torization)                                                     
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d (G  (None, 128)              0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1,280,129
Trainable params: 1,280,129
N

In [27]:
model_1.compile(
    loss="binary_crossentropy",
    optimizer=tf.keras.optimizers.Adam(),
    metrics=["accuracy"]
)

In [28]:
model_1_history = model_1.fit(
    x=train_sentences,
    y=train_labels,
    epochs=5,
    validation_data=(val_sentences, val_labels),
    callbacks=[create_tensorboard_callback(dir_name=SAVE_DIR, experiment_name="model_1_dense")]
)

Saving TensorBoard log files to: model_logs/model_1_dense/20220707-063511
Epoch 1/5
215/215 [==============================] - 7s 10ms/step - loss: 0.6123 - accuracy: 0.6938 - val_loss: 0.5375 - val_accuracy: 0.7559
Epoch 2/5
215/215 [==============================] - 2s 9ms/step - loss: 0.4435 - accuracy: 0.8161 - val_loss: 0.4746 - val_accuracy: 0.7822
Epoch 3/5
215/215 [==============================] - 2s 9ms/step - loss: 0.3480 - accuracy: 0.8608 - val_loss: 0.4637 - val_accuracy: 0.7913
Epoch 4/5
215/215 [==============================] - 2s 8ms/step - loss: 0.2860 - accuracy: 0.8892 - val_loss: 0.4622 - val_accuracy: 0.7861
Epoch 5/5
215/215 [==============================] - 2s 8ms/step - loss: 0.2390 - accuracy: 0.9102 - val_loss: 0.4798 - val_accuracy: 0.7808


In [29]:
model_1.evaluate(val_sentences, val_labels)

24/24 [==============================] - 0s 5ms/step - loss: 0.4798 - accuracy: 0.7808


[0.4798296093940735, 0.7808399200439453]

In [30]:
model_1_pred_probs = model_1.predict(val_sentences)
model_1_pred_probs

array([[0.3548926 ],
       [0.81275797],
       [0.99774384],
       [0.1094776 ],
       [0.1083597 ],
       [0.94366467],
       [0.9206508 ],
       [0.9933888 ],
       [0.96113694],
       [0.29422197],
       [0.11259472],
       [0.7146368 ],
       [0.05293193],
       [0.18360744],
       [0.00487619],
       [0.12589787],
       [0.03604896],
       [0.07311025],
       [0.23537305],
       [0.523861  ],
       [0.91697174],
       [0.04538388],
       [0.43450508],
       [0.08238105],
       [0.95837045],
       [0.9991086 ],
       [0.03773709],
       [0.06168699],
       [0.02680916],
       [0.19293283],
       [0.568274  ],
       [0.24528621],
       [0.46387312],
       [0.20802572],
       [0.49955717],
       [0.06786716],
       [0.9946108 ],
       [0.15867817],
       [0.03608301],
       [0.9985348 ],
       [0.19043502],
       [0.01987883],
       [0.34128717],
       [0.05650873],
       [0.68357784],
       [0.9829779 ],
       [0.29139122],
       [0.932

In [31]:
model_1_preds = tf.squeeze(tf.round(model_1_pred_probs))
model_1_preds

<tf.Tensor: shape=(762,), dtype=float32, numpy=
array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 1., 1., 0., 0., 0., 1., 1., 0., 0., 0., 0., 1., 0., 0., 0.,
       0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 1., 1., 0., 1., 0., 1., 0.,
       0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 1., 0., 1., 1., 1., 0.,
       0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 1., 0., 0., 1., 0., 0.,
       0., 0., 0., 1., 1., 1., 0., 1., 0., 1., 1., 1., 1., 1., 1., 1., 0.,
       0., 1., 1., 0., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 1.,
       0., 1., 0., 0., 1., 0., 0., 1., 0., 0., 1., 0., 1., 1., 1., 1., 0.,
       1., 0., 0., 0., 1., 0., 0., 1., 0., 1., 0., 0., 0., 1., 0., 0., 0.,
       0., 0., 1., 1., 0., 1., 0., 1., 0., 0., 1., 0., 0., 1., 0., 1., 0.,
       1., 1., 1., 0., 1., 0., 0., 0., 1., 1., 0., 1., 1., 1., 1., 0., 0.,
       1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 1., 0.,
       0., 1., 1., 1., 1., 1., 0., 1., 0., 0., 0., 0

In [32]:
model_1_results = calculate_results(val_labels, model_1_preds)
model_1_results, baseline_results

({'accuracy': 78.08398950131233,
  'f1': 0.777681907957685,
  'precision': 0.7856875884556138,
  'recall': 0.7808398950131233},
 {'accuracy': 79.26509186351706,
  'f1': 0.7862189758049549,
  'precision': 0.8111390004213173,
  'recall': 0.7926509186351706})

In [33]:
embed_weights = model_1.get_layer("embedding").get_weights()[0]
embed_weights.shape

(10000, 128)

# Model 2: LSTM Model

In [34]:
from tensorflow.keras import layers

inputs = layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.LSTM(64, return_sequences=True)(x)
x = layers.LSTM(64)(x)
x = layers.Dense(64, activation="relu")(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model_2 = tf.keras.Model(inputs, outputs, name="model_2_LSTM")

In [35]:
model_2.compile(
    loss="binary_crossentropy",
    optimizer=tf.keras.optimizers.Adam(),
    metrics=["accuracy"]
)

In [36]:
model_2_history = model_2.fit(
    train_sentences,
    train_labels,
    epochs=5,
    validation_data=(val_sentences, val_labels),
    callbacks=[create_tensorboard_callback(SAVE_DIR, "model_2_LSTM")]
)

Saving TensorBoard log files to: model_logs/model_2_LSTM/20220707-063536
Epoch 1/5
215/215 [==============================] - 16s 21ms/step - loss: 0.2223 - accuracy: 0.9229 - val_loss: 0.6159 - val_accuracy: 0.7795
Epoch 2/5
215/215 [==============================] - 2s 11ms/step - loss: 0.1594 - accuracy: 0.9400 - val_loss: 0.6595 - val_accuracy: 0.7822
Epoch 3/5
215/215 [==============================] - 2s 8ms/step - loss: 0.1311 - accuracy: 0.9520 - val_loss: 0.6208 - val_accuracy: 0.7717
Epoch 4/5
215/215 [==============================] - 2s 8ms/step - loss: 0.1090 - accuracy: 0.9566 - val_loss: 0.7517 - val_accuracy: 0.7743
Epoch 5/5
215/215 [==============================] - 2s 8ms/step - loss: 0.0802 - accuracy: 0.9672 - val_loss: 1.1320 - val_accuracy: 0.7717


In [37]:
model_2_pred_probs = model_2.predict(val_sentences)
model_2_pred_probs[:10]

array([[2.2254833e-03],
       [7.8139675e-01],
       [9.9996424e-01],
       [1.2614611e-02],
       [5.4340591e-05],
       [9.9969566e-01],
       [9.6614230e-01],
       [9.9997699e-01],
       [9.9996305e-01],
       [4.2008683e-01]], dtype=float32)

In [38]:
model_2_preds = tf.squeeze(tf.round(model_2_pred_probs))
model_2_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0.], dtype=float32)>

In [39]:
model_2_results = calculate_results(y_true=val_labels, y_pred=model_2_preds)
model_2_results

{'accuracy': 77.16535433070865,
 'f1': 0.7688960790251899,
 'precision': 0.7747861668850706,
 'recall': 0.7716535433070866}

# Model 3: GRU Model

In [40]:
from tensorflow.keras import layers

inputs = layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = embedding(x)
#x = layers.GRU(64, return_sequences=True)(x)
#x = layers.LSTM(64, return_sequences=True)(x)
x = layers.GRU(64)(x)
x = layers.Dense(64, activation="relu")(x)
outputs = layers.Dense(1, activation="sigmoid")(x)

model_3 = tf.keras.Model(inputs, outputs, name="model_3_GRU")

In [41]:
model_3.compile(
    loss="binary_crossentropy",
    optimizer=tf.keras.optimizers.Adam(),
    metrics=["accuracy"]
)

In [42]:
model_3_history = model_3.fit(
    train_sentences,
    train_labels,
    epochs=5,
    validation_data=(val_sentences, val_labels),
    callbacks=[create_tensorboard_callback(SAVE_DIR, "model_2_GRU")]
)

Saving TensorBoard log files to: model_logs/model_2_GRU/20220707-063605
Epoch 1/5
215/215 [==============================] - 3s 8ms/step - loss: 0.1592 - accuracy: 0.9390 - val_loss: 0.6300 - val_accuracy: 0.7782
Epoch 2/5
215/215 [==============================] - 1s 6ms/step - loss: 0.0818 - accuracy: 0.9704 - val_loss: 0.8333 - val_accuracy: 0.7717
Epoch 3/5
215/215 [==============================] - 1s 6ms/step - loss: 0.0694 - accuracy: 0.9715 - val_loss: 0.9520 - val_accuracy: 0.7730
Epoch 4/5
215/215 [==============================] - 2s 8ms/step - loss: 0.0552 - accuracy: 0.9774 - val_loss: 1.4104 - val_accuracy: 0.7769
Epoch 5/5
215/215 [==============================] - 2s 8ms/step - loss: 0.0517 - accuracy: 0.9771 - val_loss: 1.2938 - val_accuracy: 0.7677


In [43]:
model_3_pred_probs = model_3.predict(val_sentences)
model_3_pred_probs[:10]

array([[1.3237021e-02],
       [8.2805437e-01],
       [9.9999547e-01],
       [3.9076250e-02],
       [1.2887521e-05],
       [9.9997807e-01],
       [9.8104340e-01],
       [9.9999845e-01],
       [9.9999738e-01],
       [6.8230408e-01]], dtype=float32)

In [44]:
model_3_preds = tf.squeeze(tf.round(model_3_pred_probs))
model_3_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 1.], dtype=float32)>

In [45]:
model_3_results = calculate_results(y_true=val_labels, y_pred=model_3_preds)
model_3_results

{'accuracy': 76.77165354330708,
 'f1': 0.7656749923220023,
 'precision': 0.7691343474319641,
 'recall': 0.7677165354330708}

In [46]:
baseline_results

{'accuracy': 79.26509186351706,
 'f1': 0.7862189758049549,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706}

# Model 4: Bi-Directional RNN Model

In [47]:
from tensorflow.keras import layers

inputs = layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = embedding(x)
# x = layers.Bidirectional(layers.LSTM(64, return_sequences=True))(x)
x = layers.Bidirectional(layers.LSTM(64))(x)
x = layers.Dense(64, activation="relu")(x)
outputs = layers.Dense(1, activation="sigmoid")(x)

model_4 = tf.keras.Model(inputs, outputs, name="model_4_Bidirectional")

In [48]:
model_4.compile(
    loss="binary_crossentropy",
    optimizer=tf.keras.optimizers.Adam(),
    metrics=["accuracy"]
)

In [49]:
model_4_history = model_4.fit(
    train_sentences,
    train_labels,
    epochs=5,
    validation_data=(val_sentences, val_labels),
    callbacks=[create_tensorboard_callback(SAVE_DIR, "model_4_GRU")]
)

Saving TensorBoard log files to: model_logs/model_4_GRU/20220707-063620
Epoch 1/5
215/215 [==============================] - 10s 18ms/step - loss: 0.1058 - accuracy: 0.9672 - val_loss: 1.1832 - val_accuracy: 0.7717
Epoch 2/5
215/215 [==============================] - 3s 12ms/step - loss: 0.0513 - accuracy: 0.9768 - val_loss: 1.0909 - val_accuracy: 0.7703
Epoch 3/5
215/215 [==============================] - 3s 13ms/step - loss: 0.0493 - accuracy: 0.9766 - val_loss: 1.0699 - val_accuracy: 0.7638
Epoch 4/5
215/215 [==============================] - 3s 12ms/step - loss: 0.0426 - accuracy: 0.9793 - val_loss: 1.7034 - val_accuracy: 0.7651
Epoch 5/5
215/215 [==============================] - 2s 9ms/step - loss: 0.0399 - accuracy: 0.9797 - val_loss: 1.5510 - val_accuracy: 0.7730


In [50]:
model_4_pred_probs = model_4.predict(val_sentences)
model_4_pred_probs[:10]

array([[2.6807063e-03],
       [7.8552288e-01],
       [9.9998736e-01],
       [2.6989686e-01],
       [8.0103437e-06],
       [9.9986851e-01],
       [8.3538300e-01],
       [9.9999928e-01],
       [9.9999619e-01],
       [9.9012589e-01]], dtype=float32)

In [51]:
model_4_preds = tf.squeeze(tf.round(model_4_pred_probs))
model_4_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 1.], dtype=float32)>

In [52]:
model_4_results = calculate_results(y_true=val_labels, y_pred=model_4_preds)
model_4_results

{'accuracy': 77.29658792650919,
 'f1': 0.7704381702262061,
 'precision': 0.775649278370019,
 'recall': 0.7729658792650919}

# Model 5: Conv1D Model

In [53]:
import tensorflow as tf
from tensorflow.keras import layers

inputs = layers.Input(shape=(1,), dtype="string")

x = text_vectorizer(inputs)
x = embedding(x)
x = layers.Conv1D(filters=64, kernel_size=5, strides=1, activation="relu", padding="valid")(x)
x = layers.GlobalMaxPool1D()(x)

outputs = layers.Dense(1, activation="sigmoid")(x)

model_5 = tf.keras.Model(inputs, outputs, name="model_5_Conv1D")

In [54]:
model_5.compile(
    loss="binary_crossentropy",
    optimizer=tf.keras.optimizers.Adam(),
    metrics=["accuracy"]
)

model_5.summary()

Model: "model_5_Conv1D"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization (TextVec  (None, 15)               0         
 torization)                                                     
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 conv1d (Conv1D)             (None, 11, 64)            41024     
                                                                 
 global_max_pooling1d (Globa  (None, 64)               0         
 lMaxPooling1D)                                                  
                                                                 
 dense_7 (Dense)             (None, 1)              

In [55]:
model_5_history = model_5.fit(
    train_sentences,
    train_labels,
    epochs=10,
    validation_data=(val_sentences, val_labels),
    callbacks=[create_tensorboard_callback(SAVE_DIR, "Conv1D")]
)

Saving TensorBoard log files to: model_logs/Conv1D/20220707-063640
Epoch 1/10
215/215 [==============================] - 9s 6ms/step - loss: 0.1249 - accuracy: 0.9607 - val_loss: 0.9010 - val_accuracy: 0.7703
Epoch 2/10
215/215 [==============================] - 1s 5ms/step - loss: 0.0761 - accuracy: 0.9727 - val_loss: 1.0352 - val_accuracy: 0.7612
Epoch 3/10
215/215 [==============================] - 1s 5ms/step - loss: 0.0633 - accuracy: 0.9749 - val_loss: 1.1485 - val_accuracy: 0.7638
Epoch 4/10
215/215 [==============================] - 1s 5ms/step - loss: 0.0550 - accuracy: 0.9784 - val_loss: 1.1314 - val_accuracy: 0.7585
Epoch 5/10
215/215 [==============================] - 1s 5ms/step - loss: 0.0511 - accuracy: 0.9777 - val_loss: 1.2094 - val_accuracy: 0.7507
Epoch 6/10
215/215 [==============================] - 1s 5ms/step - loss: 0.0465 - accuracy: 0.9812 - val_loss: 1.2862 - val_accuracy: 0.7493
Epoch 7/10
215/215 [==============================] - 1s 5ms/step - loss: 0.0479 

In [56]:
model_5_pred_probs = model_5.predict(val_sentences)
model_5_pred_probs[:10]

array([[3.3885188e-02],
       [8.5330218e-01],
       [9.9974757e-01],
       [1.8715961e-02],
       [2.3789521e-07],
       [9.7739226e-01],
       [8.4594804e-01],
       [9.9677163e-01],
       [9.9999917e-01],
       [8.4055495e-01]], dtype=float32)

In [57]:
model_5_preds = tf.squeeze(tf.round(model_5_pred_probs))
model_5_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 1.], dtype=float32)>

In [58]:
model_5_results = calculate_results(y_true=val_labels, y_pred=model_5_preds)
model_5_results

{'accuracy': 74.80314960629921,
 'f1': 0.7443128259702623,
 'precision': 0.7515951969849605,
 'recall': 0.7480314960629921}

# Model 6: Transfer Learning -> Universal Sentence Encoder

In [59]:
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers

sentence_encoder_layer = hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder/4", input_shape=[], dtype=tf.string, name="USE")

model_6 = tf.keras.Sequential([
    sentence_encoder_layer,
    layers.Dense(64, activation="relu"),
    layers.Dense(1, activation="relu")
], name="model_6_USE")

model_6.compile(
    loss="binary_crossentropy",
    optimizer=tf.keras.optimizers.Adam(),
    metrics=["accuracy"]
)

model_6_history = model_6.fit(
    train_sentences,
    train_labels,
    epochs=5,
    validation_data=(val_sentences, val_labels),
    callbacks=[create_tensorboard_callback(SAVE_DIR, "model_6_USE")]
)

model_6.summary()

Saving TensorBoard log files to: model_logs/model_6_USE/20220707-063713
Epoch 1/5
215/215 [==============================] - 6s 15ms/step - loss: 1.9746 - accuracy: 0.7145 - val_loss: 0.5190 - val_accuracy: 0.7887
Epoch 2/5
215/215 [==============================] - 2s 11ms/step - loss: 0.4728 - accuracy: 0.7980 - val_loss: 0.5829 - val_accuracy: 0.7913
Epoch 3/5
215/215 [==============================] - 2s 11ms/step - loss: 0.4316 - accuracy: 0.8148 - val_loss: 0.5352 - val_accuracy: 0.7992
Epoch 4/5
215/215 [==============================] - 3s 12ms/step - loss: 0.4089 - accuracy: 0.8181 - val_loss: 0.4857 - val_accuracy: 0.7992
Epoch 5/5
215/215 [==============================] - 2s 11ms/step - loss: 0.4021 - accuracy: 0.8174 - val_loss: 0.5985 - val_accuracy: 0.8097
Model: "model_6_USE"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 USE (KerasLayer)            (None, 512)               256797824 

In [60]:
model_6_pred_probs = model_6.predict(val_sentences)
model_6_pred_probs[:10]

array([[0.17786452],
       [0.79318625],
       [1.1768295 ],
       [0.08865446],
       [0.67727447],
       [0.51766527],
       [1.2486702 ],
       [1.0060555 ],
       [1.0640864 ],
       [0.0799831 ]], dtype=float32)

In [61]:
model_6_preds = tf.squeeze(tf.round(model_6_pred_probs))
model_6_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 1., 1., 1., 1., 1., 0.], dtype=float32)>

In [62]:
model_6_results = calculate_results(y_true=val_labels, y_pred=model_6_preds)
model_6_results

{'accuracy': 80.97112860892388,
 'f1': 0.8070998999140856,
 'precision': 0.8153550382582151,
 'recall': 0.8097112860892388}

# Model 7: Transfer Learning with 10% of the dataset

In [77]:
## THIS CELL IS WRONG
# data leakage happens here because train_test_split overlaps with this line
train_10_percent = train_df_shuffled[["text", "target"]].sample(frac=0.1, random_state=42)
train_sentences_10_percent = train_10_percent["text"].to_list()
train_labels_10_percent = train_10_percent["target"].to_list()
len(train_sentences_10_percent), len(train_labels_10_percent)

(761, 761)

In [78]:
# THIS CELL TAKES FROM THE TRAIN_SENTENCES SO NO "DOUBLE DIPPING" OCCURS
train_sentences_10_percent = train_sentences[:int(0.1 * len(train_sentences))]
train_labels_10_percent = train_labels[:int(0.1 * len(train_sentences))]

In [79]:
train_10_percent["target"].value_counts()

0    413
1    348
Name: target, dtype: int64

In [80]:
train_df_shuffled["target"].value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [81]:
model_7 = tf.keras.models.clone_model(model_6)

model_7.compile(
    loss="binary_crossentropy",
    optimizer=tf.keras.optimizers.Adam(),
    metrics=["accuracy"]
)

model_7_history = model_7.fit(
    train_sentences_10_percent,
    train_labels_10_percent,
    epochs=5,
    validation_data=(val_sentences, val_labels),
    callbacks=[create_tensorboard_callback(SAVE_DIR, "10_percent_of_the_data")]
)

Saving TensorBoard log files to: model_logs/10_percent_of_the_data/20220707-065506
Epoch 1/5
22/22 [==============================] - 3s 43ms/step - loss: 1.1455 - accuracy: 0.5927 - val_loss: 0.6613 - val_accuracy: 0.5591
Epoch 2/5
22/22 [==============================] - 1s 25ms/step - loss: 0.5652 - accuracy: 0.7139 - val_loss: 0.5491 - val_accuracy: 0.7638
Epoch 3/5
22/22 [==============================] - 1s 26ms/step - loss: 0.4917 - accuracy: 0.8117 - val_loss: 0.5075 - val_accuracy: 0.7690
Epoch 4/5
22/22 [==============================] - 0s 21ms/step - loss: 0.4443 - accuracy: 0.8307 - val_loss: 0.5002 - val_accuracy: 0.7808
Epoch 5/5
22/22 [==============================] - 1s 26ms/step - loss: 0.4053 - accuracy: 0.8394 - val_loss: 0.4871 - val_accuracy: 0.7861


In [82]:
model_7_pred_probs = model_7.predict(val_sentences)
model_7_pred_probs[:10]

array([[0.28710306],
       [0.59187686],
       [1.088603  ],
       [0.15764093],
       [0.5667564 ],
       [0.6619388 ],
       [1.0024534 ],
       [0.92925084],
       [0.94888556],
       [0.18250988]], dtype=float32)

In [83]:
model_7_preds = tf.squeeze(tf.round(model_7_pred_probs))
model_7_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 1., 1., 1., 1., 1., 0.], dtype=float32)>

In [84]:
model_7_results = calculate_results(y_true=val_labels, y_pred=model_7_preds)
model_7_results

{'accuracy': 78.60892388451444,
 'f1': 0.7846591268156005,
 'precision': 0.7870194413148897,
 'recall': 0.7860892388451444}